# CreateRefRangesPlugin

This plugin divides a genome into genic and intergenic regions for use as reference ranges when building a Practical Haplotype Graph. To do so, it starts with a list of genes. The boundaries of the genic regions are initially at the start and end of each gene's CDS. Then, these regions are expanded outwards until a high-coverage sequence is found. Overlapping regions are merged, and intergenic regions fill in the gaps between genic regions. Finally, both genic and intergenic regions may be subdivided based on length or divergence of haplotypes. The goal is to create reference ranges where the contents of each range are very likely to be inherited as a unit, and with crossover during recombination more likely to occur at the boundaries between reference ranges. Final reference ranges are output in BED file format.

This notebook assumes you will be running the plugin within Docker. The image maizegenetics/phg (version 1.0 or later) contains the necessary dependencies, and is available on [DockerHub](https://hub.docker.com/r/maizegenetics/phg). 

## Input Files

- reference genome (fasta format). All subsequent files should use this genome as their reference. 
- gene annotations (gff3 format). At minimum must contain genes or CDS. These genes will define your genic regions.
- aligned assemblies (gvcf format). See AssemblyMAFFromAnchorwavePlugin and MAFToGVCFPlugin for more details. These assemblies will be used to estimate the divergence within each reference range and determine if a range neds to be split.
- genome coverage (wiggle format). Wiggle files describing the coverage of the aligned assemblies (above) over the reference genome. Either coverage or identity may be used. One file is required for each reference chromosome.
- Optionally, a second set of wiggle files may be provided to add more diverse and/or cross-species coverage information to further restrict the sites we consider as high-coverage. If secondary wiggle files are used, a site must have a sufficiently high coverage in both sets to be considered for a region boundary.

## Output Files

- reference ranges (bed format). The final reference ranges.
- splitWithCounts.txt (text file). Records the results of splitting reference ranges with haplotype number for each
- gene ranges (text file, optional). Records the results of expanding the boundaries of each genic region. A value of -1 for totalExpansion indicates that a window with sufficiently high coverage could not be found for that gene, and that the genic range was dropped. The region will still be covered by an intergenic range. 

## Parameters

Required parameters:
- wiggleDir: the name of the directory containing the wiggle files for determining coverage
- gffFile: the GFF file holding the genic regions
- outputBedFile: output reference ranges
- refGenome: the reference genome fasta
- vcfdir: The directory containing all the gvcf files to be used for haplotype clustering. Files are expected to be bgzipped and indexed

Optional and default parameters:
- secondaryWiggleDir: name of the directory containing the wiggle files for determining coverage using secondaryMinCover (below).
- gffFeatureType: (CDS) the feature type to use from the GFF file. Can be either CDS or gene
- minCover: (25) the minimum allowed coverage count for determining if a bp is conserved or not
- secondaryMinCover: (-1) the minimum allowed coverage count for determining if a bp is conserved or not for the secondary wiggle files
- windowSize: (10) the number of bps which must flank the current site in both directions when determining region boundaries. A window of 2 * windowSize + 1 must be conserved
- intergenicStepSize: (50000) the number of bps that the intergenic split algorithm will use to step between cut sites
- maxSearchWindow: (10000) the number of bps to check when expanding genic region boundaries
- outputGeneRanges: optional output file name for the gene ranges created y finding conserved breakpoints near genes from the gff file. If no name is supplied this file will not be written
- useSecondaryForIntergenic: (false) flag for using the secondary coverage set when finding intergenic breakpoints
- mxDiv: (0.0001) the maximum divergence allowed in the clustering step used to determine haplotype number. Haplotype number is used with the maxClusters parameter to decide when to split genic ranges
- minLength: (1000) when attempting to subdivide genic ranges, the smallest allowable subdivision length is minLength base pairs
- maxClusters: (10) When attempting to subdivide genic ranges, only genic ranges with more than maxClusters haplotypes after clustering will be subdivided
- nThreads: (10) the number of threads that will be used for subdividing genic ranges

In [1]:
###########
# EDIT ME #
###########

# working directory
working_dir = "/workdir/ahb232/phg_sorghum_apr2023/"

# docker command. Usually "docker" but should be "docker1" on biohpc
DOCKER = "docker1"

# docker image version
DOCKER_VERSION = "biohpc_ahb232/phgdev"

# config file path, relative to working_dir
config_file = "/phg/config_refrange_tests.txt"

# log file path, relative to working_dir. If set to "" log will print to this notebook
log_file =  "/refrange_tests/create_refranges_count_kmers_false_size_7_log.txt" 

In [ ]:
# RUN BUT DO NOT EDIT #

CONFIG = "/phg/" + config_file

TO_LOG = ""

if (log_file != ""):
    TO_LOG = " > " + log_file

In [ ]:
! {DOCKER} run --name create_ref_ranges --rm \
    -v {working_dir}/:/phg/ \
    -t {DOCKER_VERSION} \
    /tassel-5-standalone/run_pipeline.pl -Xmx50G -debug -configParameters {CONFIG} \
    -CreateRefRangesPlugin -endPlugin {TO_LOG}